# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have <mark>one or more teams from the "Big 4"</mark>: NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in <mark>two ordered lists of values</mark>, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be <mark>more teams than the assert statements test</mark>, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
from IPython.display import display
# %%capture
#RUN FIRST, installs a missing library
# import sys
# !{sys.executable} -m pip install lxml==4.4.1

In [2]:
def city_cleaner() : 
    # Cleaning cities data
    cities=pd.read_html("assets/wikipedia_data.html")[1].iloc[:-1,[0,3,5,6,7,8]].replace([r'\[note \d+\]', '—'], '', regex=True)
    cities.rename(columns={cities.columns[1] : 'Population'}, inplace=True)
    cities.replace('', np.NaN, inplace=True)
    cities.loc[13, 'NFL'] = np.NaN
    cities = cities.astype({'Population' : int })
    return cities

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_cleaner() :
     # Cleaning the nhl data
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df.year == 2018]
    nhl_df = nhl_df.loc[:, ['team','W', 'L', 'year']].drop(index= [0, 9, 18, 26]).reset_index(drop=True)
    nhl_df['team'] = nhl_df['team'].str.replace('*', '')
    nhl_df['Metropolitan area'] = np.NaN
    nhl_df['last word'] = nhl_df['team'].str.extract(r'\s(\w+$)')
    nhl_df = nhl_df.astype({'W' : int, 'L' : int})
    
    # Mapping Metropolitan area names to each team
    def nhl_city_identifier (row, all_cities = city_cleaner()) : 
        mask = all_cities['NHL'].str.contains(row['last word'], na=False)
        row['Metropolitan area'] = all_cities[mask].squeeze().loc['Metropolitan area']
        return row
    nhl_df = nhl_df.apply(nhl_city_identifier, axis='columns').drop(columns= ['last word'])
    return nhl_df

def nhl_correlation():
       
    cities = city_cleaner()
    
    nhl_df = nhl_cleaner()

    # grouping on metropolitan areas and calculating the ratio
    new_df = nhl_df.groupby('Metropolitan area').agg({'W' : np.sum, 'L' : np.sum})
    new_df['nhl_ratio'] = new_df['W'] / (new_df['W'] + new_df['L'])
    new_df.drop(columns=['W', 'L'], inplace=True)
    
    # adding the ration column to the cities' dataframe
    cities = cities.merge(new_df, left_on='Metropolitan area', right_index=True)
    display(cities)
    
    population_by_region = cities['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = cities['nhl_ratio'].to_list() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [4]:
nhl_correlation()

,Metropolitan area,Population,NFL,MLB,NBA,NHL,nhl_ratio
0,New York City,20153634,Giants Jets,Yankees Mets,Knicks Nets,Rangers Islanders Devils,0.518349
1,Los Angeles,13310447,Rams Chargers,Dodgers Angels,Lakers Clippers,Kings Ducks,0.622378
2,San Francisco Bay Area,6657982,49ers Raiders,Giants Athletics,Warriors,Sharks,0.625000
3,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks,0.458333
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars,0.567568
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals,0.653333
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers,0.617647
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins,0.714286
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild,0.633803
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche,0.589041


PearsonRResult(statistic=0.012308996455744278, pvalue=0.9504308637909508)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_cleaner() :
    # Cleaning the nba dataframe
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df.year == 2018]
    nba_df['team'] = nba_df['team'].replace(r'\*?\s\(\d+\)', '', regex=True)
    nba_df = nba_df.loc[:, ['team', 'W', 'L']]
    nba_df = nba_df.astype({'W' : int, 'L' : int})
    nba_df['last word'] = nba_df['team'].str.extract(r'\s(\w+$)')
    nba_df['Metropolitan area'] = np.NaN
    
    # Mapping Metropolitan area names to each team
    def nba_city_identifier (row, all_cities = city_cleaner()) : 
        mask = all_cities['NBA'].str.contains(row['last word'], na=False)
        row['Metropolitan area'] = all_cities[mask].squeeze().loc['Metropolitan area']
        return row
    nba_df = nba_df.apply(nba_city_identifier, axis='columns').drop(columns=['last word'])
    return nba_df

def nba_correlation():
    
    cities = city_cleaner()
    
    nba_df = nba_cleaner()
    
    # grouping on metropolitan areas and calculating the ratio
    new_df = nba_df.groupby('Metropolitan area').agg({'W' : np.sum, 'L' : np.sum})
    new_df['nba_ratio'] = new_df['W'] / (new_df['W'] + new_df['L'])
    new_df.drop(columns=['W', 'L'], inplace=True)
    
    # Adding the ration column to the cities' dataframe
    cities = cities.merge(new_df, left_on='Metropolitan area', right_index=True)
    population_by_region = cities['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = cities['nba_ratio'].to_list() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
   
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_cleaner() : 
    
    # Cleaning mlb data
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df.loc[mlb_df.year == 2018,['team', 'W', 'L']]
    mlb_df['last word'] = mlb_df['team'].str.extract(r'\s(\w+$)')
    mlb_df.loc[[0, 8], 'last word'] = 'Red Sox', 'White Sox'
    
    # Mapping Metropolitan area names to each team
    def mlb_city_identifier (row, all_cities = city_cleaner()) : 
        mask = all_cities['MLB'].str.contains(row['last word'], na=False)
        row['Metropolitan area'] = all_cities[mask].squeeze().loc['Metropolitan area']
        return row
    mlb_df = mlb_df.apply(mlb_city_identifier, axis='columns').drop(columns= ['last word'])
    return mlb_df

def mlb_correlation(): 
    
    cities = city_cleaner()
    mlb_df = mlb_cleaner()


    # grouping on metropolitan areas and calculating the ratio
    new_df = mlb_df.groupby('Metropolitan area').agg({'W' : np.sum, 'L' : np.sum})
    new_df['mlb_ratio'] = new_df['W'] / (new_df['W'] + new_df['L'])
    new_df.drop(columns=['W', 'L'], inplace=True)
    # Adding the ration column to the cities' dataframe
    cities = cities.merge(new_df, left_on='Metropolitan area', right_index=True)
    
    population_by_region = cities['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = cities['mlb_ratio'].to_list() # pass in win/loss ratio from nba_dfzz in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_cleaner() : 
     # Cleaning nfl data
    nfl_df=pd.read_csv("assets/nfl.csv")
    mask1 = nfl_df['year'] == 2018
    mask2 = nfl_df['team'].str.contains('AFC|NFC')
    nfl_df = nfl_df[mask1 & ~mask2].loc[:,['team', 'W', 'L']]
    nfl_df['team'].replace([r'\*', r'\+'], '', regex= True, inplace=True)
    nfl_df['last word'] = nfl_df['team'].str.extract(r'\s(\w+)$')
    nfl_df = nfl_df.astype({'W' : float, 'L' : float})
    
    # Mapping Metropolitan area names to each team
    def nfl_city_identifier (row, all_cities = city_cleaner()) : 
        mask = all_cities['NFL'].str.contains(row['last word'], na=False)
        row['Metropolitan area'] = all_cities[mask].squeeze().loc['Metropolitan area']
        return row
    nfl_df = nfl_df.apply(nfl_city_identifier, axis='columns').drop(columns = ['last word'])
    return nfl_df

def nfl_correlation(): 
    
   

    #Creating the city data
    cities = city_cleaner()
    nfl_df = nfl_cleaner()

    # grouping on metropolitan areas and calculating the ratio
    new_df = nfl_df.groupby('Metropolitan area').agg({'W' : np.sum, 'L' : np.sum})
    new_df['nfl_ratio'] = new_df['W'] / (new_df['W'] + new_df['L'])
    new_df.drop(columns=['W', 'L'], inplace=True)
    # Adding the ration column to the cities' dataframe
    cities = cities.merge(new_df, left_on='Metropolitan area', right_index=True)


    population_by_region = cities['Population'].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = cities['nfl_ratio'].to_list() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [8]:
mlb_correlation()

PearsonRResult(statistic=0.15052304487104848, pvalue=0.4629669779770919)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=mlb_cleaner()
nhl_df= nhl_cleaner()
nba_df=nba_cleaner()
nfl_df=nfl_cleaner()

cities=city_cleaner()        
def sports_team_performance():
   

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    def df_groupper(df) : 
        grouped = df.groupby('Metropolitan area').agg({'W' : np.sum, 'L' : np.sum})
        # name = name + '_ratio'
        grouped['ratio'] = grouped['W'] / (grouped['W'] + grouped['L'])
        grouped.drop(columns= ['W', 'L'], inplace= True)
        return grouped

    # Possible solution 
    sports_dict = {'MLB' : mlb_df, 'NBA' : nba_df, 'NHL' : nhl_df, 'NFL' : nfl_df}

    for ind in p_values.index : 
        for col in p_values.columns : 
            if col == ind : 
                continue
            else : 
                right_df = df_groupper(sports_dict[ind])
                left_df = df_groupper(sports_dict[col])

                merged = pd.merge(right_df, left_df, on='Metropolitan area')
                p_values.loc[ind, col] = stats.ttest_rel(merged['ratio_x'], merged['ratio_y'])[1]




    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [16]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN


In [17]:
np.arange(6)

array([0, 1, 2, 3, 4, 5])

In [27]:
import numpy as np

a = np.arange(8)
b = a[4:6]
b[:] = 40
c = a[4] + a[6]
a

array([ 0,  1,  2,  3, 40, 40,  6,  7])

In [28]:
bool(re.match('a', 'amir'))

True